# Hello!
Please read this instructions carefully and press the button below to continue.
- instruction 1
- instruction 2

Below example shows how to move to page B by click the button on the page A.

In [1]:
import ipywidgets as widgets
from IPython.display import display, IFrame, update_display

page_a_content = widgets.Label('Welcome to Page A!')
page_b_content = widgets.Label('This is Page B!')
button_start = widgets.Button(description='Go to Page B')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()  # Clear the previous output
        display(page_b_content)  # Display only the content for Page B

button_start.on_click(on_button_clicked)

# Initially display Page A content and the button
with output:
    display(page_a_content, button_start)

# Display the output widget itself
display(output)


Output()

## Question Solving Session

In [2]:
import Casino as c

# Parameters
player_money = 10
host_reward = 3
bet_lower_limit = 2
bet_upper_limit = 4

# Environment
casino = c.Casino(host_reward, bet_lower_limit, bet_upper_limit)
player = c.Player(player_money, casino=casino)
agents = [c.BraggerAgent(casino), c.CowardAgent(casino), c.RandomAgent(casino), c.SmartAgent(casino)]
playthrough = c.Playthrough(casino, player, agents, True)

def reset_env():
    global casino, player, agents, playthrough
    casino = c.Casino(host_reward, bet_lower_limit, bet_upper_limit)
    player = c.Player(player_money, casino=casino)
    agents = [c.BraggerAgent(casino), c.CowardAgent(casino), c.RandomAgent(casino), c.SmartAgent(casino)]
    playthrough = c.Playthrough(casino, player, agents, True)

In [3]:
def make_html_line(lines:list):
    html = "<p>"
    for line in lines:
        html += line + "<br>"
    html += "</p>"
    return html

def make_html_paragraph(line:str):
    paragraph_html = """
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {
                font-family: Arial, sans-serif;
                font-size: 14px;
            }
        </style>
    </head>
    <body>"""
    para = make_html_line(line.split("\n"))
    paragraph_html_end = """
    </body>
    </html>
    """
    return paragraph_html + para + paragraph_html_end


In [5]:
import time
import threading
reset_env()

page_a_content = widgets.HTML(make_html_paragraph(playthrough.get_intro_str()))

button_continue = widgets.Button(description='Continue')
button_start = widgets.Button(description='Next')
output = widgets.Output()
ui = widgets.VBox()

round_number = 0
bet = None
choice = None

main_handle = None

timer_widget = widgets.FloatProgress(
    value=0,
    min=0,
    max=30,
    description='Time Limit:',
    bar_style='info',
    orientation='horizontal',
    style={'bar_color': "green"}
)
time_begin = 0
timer_start = False

# get color from green to red
def get_color(value:float, min:float, max:float):
    return f"#" + f"{int(255 * (value - min) / (max - min)) :02x}" + f"{int(255 * (max - value) / (max - min)) :02x}" + f"00"

def on_button_continue_clicked(b):
    global round_number
    global main_handle
    global timer_start
    output.clear_output()  # Clear the previous output
    timer_start = False
    bet_value = bet.value
    if choice.value == "High":
        choice_value = 1
    elif choice.value == "Low":
        choice_value = 0
    playthrough.play_round(round_number, bet_value, choice_value)
    page_round_result_content = widgets.HTML(make_html_paragraph(playthrough.get_round_result_str(round_number)))
    round_number += 1
    if round_number >= playthrough.get_rounds():
        page_conclusion_content = widgets.HTML(make_html_paragraph(playthrough.get_conclusion_str()))
        ui.children = [page_round_result_content, page_conclusion_content]
    else:
        ui.children = [page_round_result_content,button_start]
    

def on_button_clicked(b):
    global round_number 
    global bet
    global choice
    global timer_start
    global time_begin
    global main_handle
    with output:
        output.clear_output()  # Clear the previous output
        page_round_content = widgets.HTML(make_html_paragraph(playthrough.get_round_str(round_number)))
        bets = player.get_availiable_bets()
        bet = widgets.BoundedIntText(
            value=bets[0],
            min=min(bets),
            max=max(bets),
            step=1,
            description='Bet:',
            disabled=False
        )
        choice = widgets.RadioButtons(
            options=['High', 'Low'],
            description='Choice:',
            disabled=False
        )
        time_begin = time.time()
        timer_widget.value = 0
        time_limit = playthrough.get_round_time_limit(round_number)
        timer_widget.max = time_limit
        timer_start = True
        ui.children = [page_round_content, bet, choice, timer_widget, button_continue]
        def update_time():
            global time_begin
            global button_continue
            button_continue.disabled = True
            button_continue.description = "Thinking..."
            while timer_start == False:
                time.sleep(0.1)
            while timer_widget.value < timer_widget.max:
                #timer_widget.style={'bar_color': get_color(timer_widget.value, timer_widget.min, timer_widget.max)}
                if timer_widget.value >= 5:
                    button_continue.disabled = False
                    button_continue.description = "Bet!"
                timer_widget.value = time.time() - time_begin
                time.sleep(0.1)
                
                if timer_start == False:
                    return

            button_continue.click()

        thread = threading.Thread(target=update_time)
        thread.start()
        main_handle = display(ui,display_id="main")


button_continue.on_click(on_button_continue_clicked)
button_start.on_click(on_button_clicked)


# Initially display Page A content and the button
ui.children = [page_a_content, button_start]
display(ui)
